In [1]:
import os 
os.environ["https_proxy"] = "http://xen03.iitd.ac.in:3128"
os.environ["http_proxy"] = "http://xen03.iitd.ac.in:3128"

In [2]:
# ping to google
import requests
try:
    response = requests.get("https://www.google.com", timeout=5)
    if response.status_code == 200:
        print("Internet connection is working.")
    else:
        print("Failed to connect to the internet.")
except requests.ConnectionError:
    print("No internet connection available.")

Internet connection is working.


In [3]:
import sys
# sys.path.append('../sae')
from sae import Sae
from utils import load_model_and_sae, extract_sae_features
from transformers import AutoModelForCausalLM, AutoTokenizer, LlamaForCausalLM
import torch
import torch.nn.functional as F
import os
import pandas as pd
import numpy as np
import time

In [4]:
# import model
model_type = 'gemma2-2b'
layer_num = 20
device = 'cpu'
model, tokenizer, sae = load_model_and_sae(model_type, layer_num, device)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
x1_raw_text = 'The film explores love and trauma through non-linear storytelling, blending magical realism with emotionally raw performances'
x1_raw = tokenizer(x1_raw_text, return_tensors="pt")['input_ids'].to(device)
x1_raw_preprocessed = tokenizer(x1_raw_text + "\nThe previous text is about", 
                                return_tensors="pt")['input_ids'].to(device)
h1_raw = model(x1_raw_preprocessed, output_hidden_states=True).hidden_states[layer_num + 1][0][-1].detach()

In [7]:
x2_raw_text = "The film explores love and trauma through non-linear storytelling, blending magical realism with emotionally raw performancesHacker Encryption implementations"
x2_raw = tokenizer(x2_raw_text, return_tensors="pt")['input_ids'].to(device)
x2_raw_preprocessed = tokenizer(x2_raw_text + "\nThe previous text is about", 
                                return_tensors="pt")['input_ids'].to(device)
h2_raw = model(x2_raw_preprocessed, output_hidden_states=True).hidden_states[layer_num + 1][0][-1].detach()

In [8]:
# save h1_raw and h2_raw for later use
torch.save(h1_raw, 'temp/h1_raw.pt')
torch.save(h2_raw, 'temp/h2_raw.pt')